In [2]:
import json
import os
import importlib
import time
from transformers import T5Tokenizer
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer, Regex
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import Metaspace, Split, WhitespaceSplit
from huggingface_hub import login

import re
import src.paper_to_equation.Generation.Equation_BaseDataset 


In [13]:
importlib.reload(src.paper_to_equation.Generation.Equation_BaseDataset)
from src.paper_to_equation.Generation.Equation_BaseDataset import BaseDataset

Confirming the default tokenizer is unsuitable

In [37]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

mml = """
<mml:mi>h</mml:mi>
<mml:mo>=</mml:mo>
<mml:mrow>
<mml:msub>
<mml:mi>h</mml:mi>
<mml:mi>c</mml:mi>
</mml:msub>
<mml:mo>+</mml:mo>
<mml:msub>
<mml:mi>h</mml:mi>
<mml:mi>g</mml:mi>
</mml:msub>
</mml:mrow>
  """

py = """
h = Symbol('h')
h_g = Symbol('h_g')
h_c = Symbol('h_c')
e = Eq(h, h_g + h_c)"""

tokens = tokenizer.tokenize(mml)
print(tokens)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


['▁', '<', 'mm', 'l', ':', 'm', 'i', '>', 'h', '<', '/', 'mm', 'l', ':', 'm', 'i', '>', '▁', '<', 'mm', 'l', ':', 'm', 'o', '>', '=', '<', '/', 'mm', 'l', ':', 'm', 'o', '>', '▁', '<', 'mm', 'l', ':', 'm', 'row', '>', '▁', '<', 'mm', 'l', ':', 'm', 'sub', '>', '▁', '<', 'mm', 'l', ':', 'm', 'i', '>', 'h', '<', '/', 'mm', 'l', ':', 'm', 'i', '>', '▁', '<', 'mm', 'l', ':', 'm', 'i', '>', 'c', '<', '/', 'mm', 'l', ':', 'm', 'i', '>', '▁', '<', '/', 'mm', 'l', ':', 'm', 'sub', '>', '▁', '<', 'mm', 'l', ':', 'm', 'o', '>', '+', '<', '/', 'mm', 'l', ':', 'm', 'o', '>', '▁', '<', 'mm', 'l', ':', 'm', 'sub', '>', '▁', '<', 'mm', 'l', ':', 'm', 'i', '>', 'h', '<', '/', 'mm', 'l', ':', 'm', 'i', '>', '▁', '<', 'mm', 'l', ':', 'm', 'i', '>', 'g', '<', '/', 'mm', 'l', ':', 'm', 'i', '>', '▁', '<', '/', 'mm', 'l', ':', 'm', 'sub', '>', '▁', '<', '/', 'mm', 'l', ':', 'm', 'row', '>']


Dataset class

In [14]:
class TokenizerDataset(BaseDataset):
    def __init__(self, num):
        super().__init__(num)

    def get_columns(self):
        return ["mathml", "python"]

    def map_atomic_tokens(self, dataset):

        tag_map = {"<mml:mo>": "<MO>", "</mml:mo>": "</MO>",
                   "<mml:mi>": "<MI>", "</mml:mi>": "</MI>",
                   "<mml:msub>": "<MSUB>", "</mml:msub>": "</MSUB>",
                   "<mml:msup>": "<MSUP>", "</mml:msup>": "</MSUP>",
                   "<mml:mrow>": "<MROW>", "</mml:mrow>": "</MROW>", 
                   "<mml:mfrac>": "<MFRAC>", "</mml:mfrac>": "</MFRAC>"}
        
        for entry in dataset:
            mathml = entry["mathml"]
            for tag, token in tag_map.items():
                mathml = mathml.replace(tag, token)
            entry["mathml"] = mathml
        
        return dataset
    
    def get_tag_list(self):
        return ["<mml:mo>", "</mml:mo>",
                "<mml:mi>", "</mml:mi>",
                "<mml:msub>", "</mml:msub>",
                "<mml:msup>", "</mml:msup>",
                "<mml:mrow>", "</mml:mrow>",
                "<mml:mfrac>", "</mml:mfrac>",
                "<mml:mtext>", "</mml:mtext>"]
    
    def extract_tags(self, data):
        tags = set()
        for entry in data:
            mathml = entry["mathml"]
            for tag in mathml:
                if tag not in tags:
                    tags.append(tag)
        return tags
        
    def data_iterator(self, batch_size):
        columns = self.get_columns()
        for i in range(0, len(self.dataset), batch_size):
            yield [f"{data[columns[0]]} {data[columns[1]]} "for data in self.dataset[i:i+batch_size]]  

In [17]:
td = TokenizerDataset(1000)
td.clear_dataset()
td.create_dataset()
data_iterator = td.data_iterator(5)

for i, data in enumerate(data_iterator):
    print(i, len(data))
    





Generating dataset: 100%|██████████| 1000/1000 [00:11<00:00, 89.13it/s]

0 5
1 5
2 5
3 5
4 5
5 5
6 5
7 5
8 5
9 5
10 5
11 5
12 5
13 5
14 5
15 5
16 5
17 5
18 5
19 5
20 5
21 5
22 5
23 5
24 5
25 5
26 5
27 5
28 5
29 5
30 5
31 5
32 5
33 5
34 5
35 5
36 5
37 5
38 5
39 5
40 5
41 5
42 5
43 5
44 5
45 5
46 5
47 5
48 5
49 5
50 5
51 5
52 5
53 5
54 5
55 5
56 5
57 5
58 5
59 5
60 5
61 5
62 5
63 5
64 5
65 5
66 5
67 5
68 5
69 5
70 5
71 5
72 5
73 5
74 5
75 5
76 5
77 5
78 5
79 5
80 5
81 5
82 5
83 5
84 5
85 5
86 5
87 5
88 5
89 5
90 5
91 5
92 5
93 5
94 5
95 5
96 5
97 5
98 5
99 5
100 5
101 5
102 5
103 5
104 5
105 5
106 5
107 5
108 5
109 5
110 5
111 5
112 5
113 5
114 5
115 5
116 5
117 5
118 5
119 5
120 5
121 5
122 5
123 5
124 5
125 5
126 5
127 5
128 5
129 5
130 5
131 5
132 5
133 5
134 5
135 5
136 5
137 5
138 5
139 5
140 5
141 5
142 5
143 5
144 5
145 5
146 5
147 5
148 5
149 5
150 5
151 5
152 5
153 5
154 5
155 5
156 5
157 5
158 5
159 5
160 5
161 5
162 5
163 5
164 5
165 5
166 5
167 5
168 5
169 5
170 5
171 5
172 5
173 5
174 5
175 5
176 5
177 5
178 5
179 5
180 5
181 5
182 5
183 5
184 5


In [22]:
tokenizer = Tokenizer(models.Unigram())
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Replace("``", '"'), normalizers.Replace("''", '"')] # don't lowercase
)
tokenizer.normalizer.normalize_str(py)

tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()
tokenizer.pre_tokenizer.pre_tokenize_str(py)



[('▁\nh', (0, 2)),
 ('▁=', (2, 4)),
 ("▁Symbol('h')\nh_g", (4, 20)),
 ('▁=', (20, 22)),
 ("▁Symbol('h_g')\nh_c", (22, 40)),
 ('▁=', (40, 42)),
 ("▁Symbol('h_c')\ne", (42, 58)),
 ('▁=', (58, 60)),
 ('▁Eq(h,', (60, 66)),
 ('▁h_g', (66, 70)),
 ('▁+', (70, 72)),
 ('▁h_c)', (72, 77))]

In [3]:
tokenizer = Tokenizer(models.BPE())                  

class MathMLPyTokenizer(pre_tokenizers.PreTokenizer):
    def __init__(self):
        pass
    
    def pre_tokenize(self, data):
        
        patterns = [
            # HTML/XML tags
            r'<[^>]+>',
            # Python string literals
            r'"""[^"]*"""', r"'''[^']*'''", r'"[^"]*"', r"'[^']*'",
            # Python keywords and operators
            r'\bdef\b', r'\bclass\b', r'\bfor\b', r'\bwhile\b', r'\bif\b', r'\belif\b', r'\belse\b',
            r'\breturn\b', r'\bimport\b', r'\bfrom\b', r'\bas\b', r'\bwith\b', r'\btry\b', r'\bexcept\b',
            # Common Python syntax elements
            r'==', r'!=', r'<=', r'>=', r'\+=', r'-=', r'\*=', r'/=', 
            r'=>', r'->',  # Function type hints and lambdas
            r'\bSymbol\b', r'\bEq\b', r'\bexp\b', r'\bsin\b', r'\bcos\b', r'\btan\b', r'\bdiff\b',
            
            # Indentation (important for Python)
            r'^\s+'
        ]

        combined_pattern = '|'.join(f'({p})' for p in patterns)
        regex = re.compile(combined_pattern, re.MULTILINE)
        
        splits = []
        for text, offset in data:
            last_end = 0
            for match in regex.finditer(text):
                start, end = match.span()
                
                if start > last_end:
                    # Add text before the special token
                    splits.append((text[last_end:start], last_end + offset))
                
                # Add the special token as a whole
                splits.append((text[start:end], start + offset))
                last_end = end
            
            if last_end < len(text):
                # Add remaining text
                splits.append((text[last_end:], last_end + offset))
                
        return splits

x = MathMLPyTokenizer()

tokens = tokenizer.tokenize(mml)
print(tokens)


TypeError: No constructor defined

In [17]:
if __name__ == "__main__":
    start1 = time.time()
    td1 = TokenizerDataset(10000)
    td1.create_dataset()
    end1 = time.time()

    start2 = time.time()
    td2 = TokenizerDataset(10000)
    td2.create_dataset_mthread()
    end2 = time.time()

    print("Time 1: ", end1 - start1)
    print("Time 2: ", end2 - start2)

Generating dataset:  92%|█████████▏| 9223/10000 [01:28<00:07, 104.64it/s] 

Time 1:  99.67798781394958
Time 2:  90.23344993591309


In [ ]:
td = TokenizerDataset(10000)
td.create_dataset()
print(len(td.dataset))
data_iterator = td.data_iterator(1000)

Generating dataset: 0it [00:00, ?it/s]

In [54]:
# Create a base tokenizer (BPE, WordPiece, or Unigram)
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

# Define patterns we want to protect during tokenization
text_patterns = [
            # HTML/XML tags
            r'<[^>]+>',
            # Python string literals
            r'"""[^"]*"""', r"'''[^']*'''", r'"[^"]*"', r"'[^']*'",
            # Python keywords and operators
            r'\bdef\b', r'\bclass\b', r'\bfor\b', r'\bwhile\b', r'\bif\b', r'\belif\b', r'\belse\b',
            r'\breturn\b', r'\bimport\b', r'\bfrom\b', r'\bas\b', r'\bwith\b', r'\btry\b', r'\bexcept\b',
            # Common Python syntax elements
            r'==', r'!=', r'<=', r'>=', r'\+=', r'-=', r'\*=', r'/=', 
            r'=>', r'->',  # Function type hints and lambdas
            r'\bSymbol\b', r'\bEq\b', r'\bexp\b', r'\bsin\b', r'\bcos\b', r'\btan\b', r'\bdiff\b', # MathML
            # Indentation (important for Python)
            r'^\s+',
            r'\s+' # Whitespace
        ]

# Set up the pre-tokenizer using Split with pattern
pattern = '|'.join(text_patterns)
split_pre_tokenizer = Split(pattern=Regex(pattern), behavior="isolated")
tokenizer.pre_tokenizer = split_pre_tokenizer

# Set up other components 
tokenizer.normalizer = Sequence([NFKC()])
tokenizer.decoder = decoders.ByteLevel()

tag_list = td.get_tag_list()
special_tokens = ["[PAD]", "[BOS]", "[EOS]", "[UNK]"] + tag_list

trainer = trainers.BpeTrainer(vocab_size=10000, special_tokens=special_tokens)
tokenizer.train_from_iterator(iterator=data_iterator, trainer=trainer)



In [26]:
hf_login_key = os.environ.get("HF_LOGIN_KEY")
login(token=hf_login_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\kyanj\.cache\huggingface\token
Login successful


In [55]:
tokenizer.save("Tokenizer_Files/mathml-py-tokenizer.json")

In [57]:
output_mathml = tokenizer.encode(td.dataset[0]["mathml"])
output_py = tokenizer.encode(td.dataset[0]["python"])
# print(output_py.tokens)
# print(output_py.tokens)
# print(output_py.ids)
# print(tokenizer.decode(output_py.ids))
output_test = tokenizer.encode(mml)
print(output_test.tokens)

['\n', '<mml:mi>', 'h', '</mml:mi>', '\n', '<mml:mo>', '=', '</mml:mo>', '\n', '<mml:mrow>', '\n', '<mml:msub>', '\n', '<mml:mi>', 'h', '</mml:mi>', '\n', '<mml:mi>', 'c', '</mml:mi>', '\n', '</mml:msub>', '\n', '<mml:mo>', '+', '</mml:mo>', '\n', '<mml:msub>', '\n', '<mml:mi>', 'h', '</mml:mi>', '\n', '<mml:mi>', 'g', '</mml:mi>', '\n', '</mml:msub>', '\n', '</mml:mrow>', '\n', ' ', ' ']


In [ ]:
test_text = """
def my_function(x, y): return x + y
class MyClass:
    def method(self): pass
import numpy as np
<math><msup><mi>x</mi><mn>2</mn></msup></math>
"""

check = tokenizer.pre_tokenizer.pre_tokenize_str(mml)
print(check)
output = tokenizer.encode(test_text)
print(output.tokens)
print(output.ids)

Matches: ['def my_function(x, y)', 'class MyClass', 'def method(self)', 'import numpy', '<math>', '<msup>', '<mi>', '</mi>', '<mn>', '</mn>', '</msup>', '</math>']
